# Monte Carlo - Unidad 2, Sesion 3 - Ejercicio

Problema: se desea estimar el volumen de una region R de [0,1]^6 definida por todos los puntos de la hiper-esfera de centro (0.45, 0.5, 0.6, 0.6, 0.5, 0.45) y radio 0.35 que ademas cumplen con las restricciones 3x1 + 7x4 <= 5; x3+x4 <= 1; x1-x2-x5+x6 >= 0

## Entrega 2 - Ejercicio 3.1

### Parte a: 

Implementar un programa que reciba como par´ametro la
cantidad de replicaciones n a realizar, y emplee Monte Carlo para
calcular (e imprimir) la estimaci´on del volumen de R, y la desviaci´on
est´andar de este estimador. Incluir c´odigo para calcular el tiempo de
c´alculo empleado por el programa. Utilizar el programa con n = 104 y
luego con n = 106 para estimar el volumen de R. Discutir si los dos
valores obtenidos parecen consistentes.
(en la sesi´on 5 se continuar´a este ejercicio).

In [24]:
import random
import math
random.seed()

def sortearPuntoRN(dim=2):
    """
    Seortea un punto en R^N dentro del hiper-cubo [0,1]^N
    """
    punto = []
    for n in range(0, dim):
        punto.append(random.uniform(0.0, 1.0))
    # end for

    return punto
# end fun sortearPuntoRN

def puntoDentroVolumen(punto, restricciones=True):
    """
    Devuelve 0 o 1 si un punto esta fuera o dentro de un cierto volumen
    """

    # Para que este dentro del volumen tiene que estar dentro de la esfera 
    # y ademas cumplir con las restricciones adicionales

    dentro = 1
    fuera = 0

    # chequeo 1 : dentro de esfera

    d = math.sqrt(
        (punto[0]-0.45)**2 +
        (punto[1]-0.5)**2 +
        (punto[2]-0.6)**2 +
        (punto[3]-0.6)**2 +
        (punto[4]-0.5)**2 +
        (punto[5]-0.45)**2 
    )

    # si la distancia es mayor al radio, esta fuera
    if (d>=0.35) : 
        return fuera

    if restricciones:
        # restriccion 1
        if 3*punto[0] + 7*punto[3] > 5:
            return fuera
        # restriccion 2
        if punto[2]+punto[3] > 1:
            return fuera
        # restriccion 3
        if punto[0]-punto[1]-punto[4]+punto[5] < 0:
            return fuera
    else:
        return dentro

    return dentro
# end fun punto dentro del volumen

sortearPuntoRN(6)

[0.9379653775496534,
 0.7134601945765607,
 0.6459610578638343,
 0.35703499057202814,
 0.46660296411519275,
 0.31095073571064336]

In [27]:
N = 10**5

# Implemento pseudocodigo Montecarlo

def MetodoMonteCarlo():
    S = 0
    for j in range(0, N):
        punto = sortearPuntoRN(6)
        if puntoDentroVolumen(punto, False):
            phi = 1
        else: 
            phi = 0
        S = S + phi
    # end for
    VolR = S / N
    VarVorR = (S/N)*(1-S/N)/(N-1)
    return (VolR, VarVorR)
# end def

VolH = math.pi**3*(0.35**6)/6

(VolR, ValVorR) = MetodoMonteCarlo()

print("Volumen = {}, Varianza = {}".format(VolR, ValVorR))
print(" ")
print("Volumen hiper esfera = {}, diferencia = {}".format(VolH, VolH-VolR))

Volumen = 0.00911, Varianza = 9.02709817098171e-08
 
Volumen hiper esfera = 0.009499628763439042, diferencia = 0.0003896287634390424


## TBW